# Gen559 modules practice: ORFs
### 2020.11.18

*For the problems in this notebook, you will need two input files that can be downloaded from Canvas and https://github.com/beliveau-lab/gen559/tree/main/notebooks: codons.txt and plasmid.fa

### Practice problem 1a 

In the cell below, write out in words the major operational steps required to solve **Practice problem 1b**.
>In the cell below, write a function that finds open reading frames (ORFs) in an input DNA sequence. Your code only needs to consider the sense / + strand and should return a list with any ORFs found and their lengths in amino acids. Alert  the user if no valid ORFs are found. Demonstrate the functionality of your code on an input sequence of your own creation. Hint: open reading frames are contiguous stretches of DNA that code for proteins using triplet codons, starting with ATG and ending with a stop codon.

*  Define function that takes a block of DNA sequence as input
*  Search input DNA sequence for the presence of start codons and in-frame stop codons
*  Record the positions of any ORFs in the input, alert user if none found
*  Return any identified ORFs and their lengths

### Practice problem 1b
In the cell below, write a function that finds open reading frames (ORFs) in an input DNA sequence. Your code only needs to consider the sense / + strand and should return a list with any ORFs found and their lengths in amino acids. Alert  the user if no valid ORFs are found. Demonstrate the functionality of your code on an input sequence of your own creation. *Hint: open reading frames are contiguous stretches of DNA that code for proteins using triplet codons, starting with ATG and ending with a stop codon*.

In [1]:
def get_ORFs(seq):
    '''Identifies ORFs in input DNA sequence and returns the positions and lengths of any found'''
    
    # Create a list to hold function return.
    output_list = []
    
    ## Iterate through input sequence, looking for valid ORFs.
    
    # Create variable to keep track of index in sequence.
    idx = 0
    
    # Loop through input DNA sequence in windows of 3 nt.
    for i in range(idx, len(seq)-2, 1):
        window = seq[i:i+3]
        
        # Check if current window is a start codon.
        if window == "ATG":
            
            # If so, search for a stop codon.
            for j in range(i, len(seq), 3):
                current_codon = seq[j:j+3]
                if current_codon in ["TGA", "TAA", "TAG"]:
                    
                    # If found, add ORF info to output list.
                    ORF_seq = seq[i:j+3]
                    AA_len = len(ORF_seq)/ 3 - 1

                    # Only report if protein is >1 AA:
                    if AA_len > 1: 
                        output_list.append('%s\t%s' % (ORF_seq, AA_len))

                    # Update idx variable.
                    idx = i
                    
                    # break loop
                    break
                                           
    # Check if output list is empty:
    if len(output_list) == 0:
        return 'Warning, no valid ORFs found!'
    
    # If not, return list of ORFs their lengths.   
    else:
        return output_list

    
# Create simple test sequences for function. seq1 has an ORF, seq2 does not.
seq1 = 'ATGCAAAAACAAGGGTAACAT'
seq2 = 'AACCAATGGAATA'
# Call the get_ORFs function on the test inputs.
print(get_ORFs(seq1))
print(get_ORFs(seq2))

['ATGCAAAAACAAGGGTAA\t5.0']
Warning, no valid ORFs found!


### Practice problem 2a 

In the cell below, write out in words the major operational steps required to solve **Practice problem 2b**.
>In the cell below, write a function that translates an ORF in an input DNA sequence. Alert  the user if no valid ORF is found. Your function can be built to expect one ORF per input sequence, and in the case of nested ORFs return the protein product of the largest ORF. Demonstrate the functionality of your code on an input sequence of your own creation.

*  Define function that takes DNA sequence, codon table as inputs
*  Build dictionary of codon:amino acid pairings
*  Check input sequence for valid ORFs
*  Process sequence and translate ORF

### Practice problem 2b
In the cell below, write a function that translates an ORF in an input DNA sequence. Alert  the user if no valid ORF is found. Your function can be built to expect one ORF per input sequence, and in the case of nested ORFs return the protein product of the largest ORF. Demonstrate the functionality of your code on an input sequence of your own creation.

In [2]:
def translate_ORF(seq, codon_table='codons.txt'):
    '''Translates an input DNA sequence using a provided codon table'''
    
    ## Check input sequence for a valid ORF.
    
    # Check that seq begins with a start codon, ends with a stop codon,
    # and that the length of the sequence is a clean multiple of 3 using modulo '%' division.
    if seq[0:3] == 'ATG' and seq[-3:] in ["TGA", "TAA", "TAG"] and len(seq) % 3 == 0:
        
        ## If true, build codon:amino acid dictionary and translate ORF.
        
        # Make list to hold output.
        protein = []
        
        # Open codon table, extract contents.
        with open(codon_table, 'r') as f:
            codon_info = [line.strip() for line in f]
        
        # Create empty dictionary to hold codon:amino acid information.
        codon_dict = {}
        
        # Populate dictionary from input codon table.
        for i in range(0, len(codon_info), 1):
            codon_dict[codon_info[i].split('\t')[0]] = codon_info[i].split('\t')[2]
        
        # Process input sequence and translate using codon_dict.
        for i in range(0, len(seq)-2, 3):
            codon_seq = seq[i:i+3]
            
            # Check if current codon is a stop codon, break loop if so.
            if codon_dict[codon_seq] == '0':
                break
            
            # Else, add the amino acid corresponding to the current codon to the protein list.
            else:
                protein.append(codon_dict[codon_seq])
            
        # Return the translated protein as a string.        
        return ''.join(protein)
            
    
    # If not true, return warning message
    else:
        return('Warning, input sequence is not a valid ORF')
    
    
# Create some simple test ORFs. ORF1 is a valid ORF, ORF2 is not.
ORF1 = 'ATGCAACCCAAATAA'
ORF2 = 'CATCATCATTAA'

# Call the translate_ORF function on the test inputs.
print(translate_ORF(ORF1))
print(translate_ORF(ORF2))

MQPK
Warning, input sequence is not a valid ORF


### Practice problem 3
Create a new Python script containing the functions from **Probem 1b** and **Problem 2b**, ORF_utils.py, in the same directory as this notebook. Import ORF_utils.py as a module, and use its functions to identify and translate any ORFs present in plasmid.fa. Print the returns from each function.

In [3]:
# Import ORF_utils module
import ORF_utils

# Open input .fa sequence and extract DNA sequence.
with open('plasmid.fa', 'r') as f:
    fasta_seq = [line.strip() for line in f][1]

# Call the get_ORFs function from ORF_utils.py.
fasta_ORFs = ORF_utils.get_ORFs(fasta_seq)

# Print result of get_ORFs function call.
print ('\n'.join(fasta_ORFs))

# Call the translate_ORF function from ORF_utils.py on the ORFs found in the plasmid.fa sequence, print each return.
for i in range(0, len(fasta_ORFs), 1):
    seq_to_translate = fasta_ORFs[i].split('\t')[0]
    print(ORF_utils.translate_ORF(seq_to_translate))

ATGTACCCATACGATGTTCCAGATTACGCTGAATTCGGAAAGAAAACCAAGCGGACAGCTGACAGTTCTTCTTCAGAGGATGAGGAGGAGTATGTTGTGGAGAAGGTGCTAGACAGGCGCGTGGTTAAGGGACAAGTGGAATATCTACTGAAGTGGAAAGGCTTTTCTGAGGAGCACAATACTTGGGAACCTGAGAAAAACTTGGATTGCCCTGAGCTAATTTCTGAATTTATGAAAAAGTATAAGAAGATGAAGGAGGGTGAAAATAATAAACCCAGGGAGAAGTCAGAAAGTAACAAGAGGAAATCCAATTTCTCAAACAGTGCCGATGACATCAAATCTAAAAAAAAGAGAGAGCAGAGCAATGATATCGCTCGGGGCTTTGAGAGAGGACTGGAACCAGAAAAGATCATTGGGGCAACAGATTCCTGTGGTGATTTAATGTTCCTAATGAAATGGAAAGACACAGATGAAGCTGACCTGGTTCTTGCAAAAGAAGCTAATGTGAAATGTCCACAAATTGTGATAGCATTTTATGAAGAGAGACTGACATGGCATGCATATCCTGAGGATGCGGAAAACAAAGAGAAAGAAACAGCAAAGAGCTAA	202.0
ATGTTCCAGATTACGCTGAATTCGGAAAGAAAACCAAGCGGACAGCTGACAGTTCTTCTTCAGAGGATGAGGAGGAGTATGTTGTGGAGAAGGTGCTAG	32.0
ATGAGGAGGAGTATGTTGTGGAGAAGGTGCTAG	10.0
ATGTTGTGGAGAAGGTGCTAG	6.0
ATGAAAAAGTATAAGAAGATGAAGGAGGGTGAAAATAATAAACCCAGGGAGAAGTCAGAAAGTAACAAGAGGAAATCCAATTTCTCAAACAGTGCCGATGACATCAAATCTAAAAAAAAGAGAGAGCAGAGCAATGATATCGCTCGGGGCTTTGAGAGAGGACTGGAACCAGAAAAGATCATTGGGGCAACAGATTCCTGTGGTGATTTAATGT